# Handle the data
First we need to load our data file. The data is in TXT format without a header line or any quotes. You can open the file with  open function and read the data lines with reader function from the csv module

In [ ]:
import csv
with open(r'/content/Iris.csv') as csvfile:
	lines = csv.reader(csvfile)
	for row in lines:
		print(', '.join(row))

Id, SepalLengthCm, SepalWidthCm, PetalLengthCm, PetalWidthCm, Species
1, 5.1, 3.5, 1.4, 0.2, Iris-setosa
2, 4.9, 3.0, 1.4, 0.2, Iris-setosa
3, 4.7, 3.2, 1.3, 0.2, Iris-setosa
4, 4.6, 3.1, 1.5, 0.2, Iris-setosa
5, 5.0, 3.6, 1.4, 0.2, Iris-setosa
6, 5.4, 3.9, 1.7, 0.4, Iris-setosa
7, 4.6, 3.4, 1.4, 0.3, Iris-setosa
8, 5.0, 3.4, 1.5, 0.2, Iris-setosa
9, 4.4, 2.9, 1.4, 0.2, Iris-setosa
10, 4.9, 3.1, 1.5, 0.1, Iris-setosa
11, 5.4, 3.7, 1.5, 0.2, Iris-setosa
12, 4.8, 3.4, 1.6, 0.2, Iris-setosa
13, 4.8, 3.0, 1.4, 0.1, Iris-setosa
14, 4.3, 3.0, 1.1, 0.1, Iris-setosa
15, 5.8, 4.0, 1.2, 0.2, Iris-setosa
16, 5.7, 4.4, 1.5, 0.4, Iris-setosa
17, 5.4, 3.9, 1.3, 0.4, Iris-setosa
18, 5.1, 3.5, 1.4, 0.3, Iris-setosa
19, 5.7, 3.8, 1.7, 0.3, Iris-setosa
20, 5.1, 3.8, 1.5, 0.3, Iris-setosa
21, 5.4, 3.4, 1.7, 0.2, Iris-setosa
22, 5.1, 3.7, 1.5, 0.4, Iris-setosa
23, 4.6, 3.6, 1.0, 0.2, Iris-setosa
24, 5.1, 3.3, 1.7, 0.5, Iris-setosa
25, 4.8, 3.4, 1.9, 0.2, Iris-setosa
26, 5.0, 3.0, 1.6, 0.2, Iris-setosa
27,

# Defining the loadDataset function
Define a function called loadDataset which loads a CSV and splits it randomly into train and test datasets using the split ratio.

In [ ]:
import csv
import random
def loadDataset(filename, split, trainingSet=[] , testSet=[]):
	with open(filename, 'r') as csvfile:
	    lines = csv.reader(csvfile)
	    dataset = list(lines)
	    for x in range(len(dataset)-1):
	        for y in range(4):
	            dataset[x][y] = float(dataset[x][y])
	        if random.random() < split:
	            trainingSet.append(dataset[x])
	        else:
	            testSet.append(dataset[x])

# Testing the loadDataset Function


In [ ]:
trainingSet=[]
testSet=[]
loadDataset(r'/content/Iris.csv', 0.66, trainingSet, testSet)
print('Train: '+ repr(len(trainingSet)))
print('Test: '+ repr(len(testSet)))

ValueError: could not convert string to float: 'Id'

In [ ]:
import csv
import random
def loadDataset(filename, split, trainingSet=[] , testSet=[]):
    with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        # Skip the header row
        next(lines, None)
        dataset = list(lines)
        for x in range(len(dataset)):
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
            if random.random() < split:
                trainingSet.append(dataset[x])
            else:
                testSet.append(dataset[x])

In [ ]:
trainingSet=[]
testSet=[]
loadDataset(r'/content/Iris.csv', 0.66, trainingSet, testSet)
print('Train: '+ repr(len(trainingSet)))
print('Test: '+ repr(len(testSet)))

Train: 103
Test: 47


# Calculating the similarity
This is required to locate the k most similar data instances in the training dataset for a given member of the test dataset and in turn make a prediction.

Given that all four flower measurements are numeric and have the same units, you can directly use the Euclidean distance measure. Eucledian Distance is the square root of the sum of the squared differences between the two arrays of numbers

In [ ]:
import math
def euclideanDistance(instance1, instance2, length):
	distance = 0
	for x in range(length):
		distance += pow((instance1[x] - instance2[x]), 2)
	return math.sqrt(distance)

# Testing the euclideanDistance Function


In [ ]:
data1 = [2, 2, 2, 'a']
data2 = [4, 4, 4, 'b']
distance = euclideanDistance(data1, data2, 3)
print('Distance: ' + repr(distance))

Distance: 3.4641016151377544


# Finding K most similar neighbors
getNeighbors function that returns k most similar neighbors from the training set for a given test instance

In [ ]:
import operator
def getNeighbors(trainingSet, testInstance, k):
	distances = []
	length = len(testInstance)-1
	for x in range(len(trainingSet)):
		dist = euclideanDistance(testInstance, trainingSet[x], length)
		distances.append((trainingSet[x], dist))
	distances.sort(key=operator.itemgetter(1))
	neighbors = []
	for x in range(k):
		neighbors.append(distances[x][0])
	return neighbors

# Testing getNeighbors function

In [ ]:
trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]
testInstance = [5, 5, 5]
k = 1
neighbors = getNeighbors(trainSet, testInstance, 1)
print(neighbors)

[[4, 4, 4, 'b']]


# Predicting response based on the neighbors
getResponse function is used for getting the majority voted response from a number of neighbors. It assumes the last attribute for each neighbor is its class.

In [ ]:
import operator
def getResponse(neighbors):
	classVotes = {}
	for x in range(len(neighbors)):
		response = neighbors[x][-1]
		if response in classVotes:
			classVotes[response] += 1
		else:
			classVotes[response] = 1
	sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
	return sortedVotes[0][0]

# Testing getResponse Function

In [ ]:
neighbors = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
response = getResponse(neighbors)
print(response)

a


# Evaluating the accuracy of prediction
This getAccuracy function sums the total correct predictions and returns the accuracy as a percentage of correct classifications.

In [ ]:
def getAccuracy(testSet, predictions):
	correct = 0
	for x in range(len(testSet)):
		if testSet[x][-1] is predictions[x]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0

# Testing getAccuracy Function

In [ ]:
testSet = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
predictions = ['a', 'a', 'a']
accuracy = getAccuracy(testSet, predictions)
print(accuracy)

66.66666666666666


# Defining the main function
Now that you have defined all the elements of the algorithm, you can tie them together with a main function.

In [ ]:
def main():
	# prepare data
	trainingSet=[]
	testSet=[]
	split = 0.67
	loadDataset(r'C:\Users\Admin\Documents\Kiran\Documents\Kiran\Accenture\Python\IBM\KNN\iris.csv', split, trainingSet, testSet)
	print('Train set: ' + repr(len(trainingSet)))
	print ('Test set: ' + repr(len(testSet)))
	# generate predictions
	predictions=[]
	k = 3
	for x in range(len(testSet)):
		neighbors = getNeighbors(trainingSet, testSet[x], k)
		result = getResponse(neighbors)
		predictions.append(result)
		print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
	accuracy = getAccuracy(testSet, predictions)
	print('Accuracy: ' + repr(accuracy) + '%')
main()

Train set: 101
Test set: 48
> predicted='setosa', actual='setosa'
> predicted='setosa', actual='setosa'
> predicted='setosa', actual='setosa'
> predicted='setosa', actual='setosa'
> predicted='setosa', actual='setosa'
> predicted='setosa', actual='setosa'
> predicted='setosa', actual='setosa'
> predicted='setosa', actual='setosa'
> predicted='setosa', actual='setosa'
> predicted='setosa', actual='setosa'
> predicted='setosa', actual='setosa'
> predicted='setosa', actual='setosa'
> predicted='setosa', actual='setosa'
> predicted='setosa', actual='setosa'
> predicted='versicolor', actual='versicolor'
> predicted='versicolor', actual='versicolor'
> predicted='versicolor', actual='versicolor'
> predicted='versicolor', actual='versicolor'
> predicted='versicolor', actual='versicolor'
> predicted='versicolor', actual='versicolor'
> predicted='versicolor', actual='versicolor'
> predicted='versicolor', actual='versicolor'
> predicted='versicolor', actual='versicolor'
> predicted='versicolor', 